In [ ]:
#1 Import or load osmnx graph

# get network data from local directory if exists, download it from OSM + gets elevation from Google API if not
street_path = Path('/Users/david/Dropbox/PhD/GitHub/Physical environment modelling/Accessibility/')
if os.path.exists(street_path):
    print('File existed.')
    # load as GeoDataFrame
    nodes = gpd.read_file(street_path/'data/sample/nodes/nodes.shp')
    edges = gpd.read_file(street_path/'data/sample/edges/edges.shp')
    nodes['x'] = nodes.geometry.x
    nodes['y'] = nodes.geometry.y
    nodes['elevation'] = nodes['elevation'].astype(float)
    nodes['osmid'] = nodes['osmid'].astype('int64')
    nodes = nodes.set_index('osmid',drop = False)
    edges['u'] = edges['from'].astype('int64')
    edges['v'] = edges['to'].astype('int64')
    edges['length'] = edges['length'].astype(float)
    edges['grade'] = edges['grade'].astype(float)
    edges['grade_abs'] = edges['grade_abs'].astype(float)
    nodes.gdf_name = 'unnamed_UTM_nodes'
    G = ox.gdfs_to_graph(nodes, edges)
else:
    G = ox.graph_from_polygon(gva,simplify = True,clean_periphery = True) #Get the graph from OSM
    # add elevation to nodes automatically, calculate edge grades, plot network
    G = ox.add_node_elevations(G, api_key=google()) #Insert your own Google API key
    G = ox.add_edge_grades(G)
    ox.save_graph_shapefile(G, filename='sample', folder=None, encoding='utf-8')
    print('Network downloaded and saved under data/sample')
    nx.set_node_attributes(G, name='elevation', values=df['elevation'].to_dict())

In [ ]:
#2 Create center of points of interest (POIs)
#Transform to geodataframe
geometry = [Point(xy) for xy in zip(df_centre['lon'], df_centre['lat'])]
# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}
gdf_centre = gpd.GeoDataFrame(df_centre, crs=crs, geometry=geometry)

gdf_centre.index = gdf_centre.index.map(int)
gdf_centre['lon'] = gdf_centre.geometry.x
gdf_centre['lat'] = gdf_centre.geometry.y
gdf_centre['id'] = gdf_centre.index+100000
gdf_centre['quantity'] = 1.0
gdf_centre['category'] = 'centre'

In [ ]:
# define some edge impedance function here
def impedance(length, grade):
    penalty = grade ** 2
    return length * penalty
# add impedance and elevation rise values to each edge in the projected graph
# use absolute value of grade in impedance function if you want to avoid uphill and downhill
for u, v, k, data in G.edges(keys=True, data=True):
    data['impedance'] = impedance(float(data['length']), float(data['grade_abs']))
    data['rise'] = float(data['length']) * float(data['grade'])

In [ ]:
people = gdf_geom_ok[['geom_id','GKODE','GKODN','geometry']].drop_duplicates().reset_index(drop = True)
people.columns = ['id','lon','lat','geometry']

In [ ]:
pois = pd.concat([people[['id','lon','lat','quantity','category','geometry']],gdf_centre[['id','lon','lat','quantity','category','geometry']]],sort = False).reset_index(drop = True)
pois.index = pois.index.map(int)

In [ ]:
from toolbox import connect_poi
import pandana as pdna

new_nodes, new_edges = connect_poi(pois, nodes, edges, key_col='id', path=None)
print("POIs snapped to network")
new_nodes['x'] = new_nodes.geometry.x
new_nodes['y'] = new_nodes.geometry.y
new_nodes['osmid'] = new_nodes['osmid'].astype('int64')
new_nodes.index = new_nodes.index.map(int)
new_edges['from'] = new_edges['from'].astype('int64')
new_edges['to'] = new_edges['to'].astype('int64')
network=pdna.Network(new_nodes["x"], new_nodes["y"], new_edges["from"], new_edges["to"],new_edges[["length"]],twoway = True)

In [ ]:
# output
poi_links = new_edges[new_edges['highway'] == 'projected_footway']
ax = edges.plot(linewidth=0.8, figsize=(18,10), label='Original Road Edges')
poi_links.plot(color='indianred', linewidth=2, ax=ax, label='New Connection Edges')
# pois[pois.category == 'participant'].plot(color='orange', marker='.', markersize=50, ax=ax, label='Cancer Screening participant')
pois[pois.category == 'centre'].plot(color='green', marker='.', markersize=80, ax=ax, label='POIs')
ax.legend(loc=2, fontsize=18)
ax.set_title('The integrated network of POIs and road network in the state of Geneva', fontsize=22);
ax.set_axis_off()

In [ ]:
network.precompute(2500)

In [ ]:
dict_density = {}
gdf_pois = pois[pois.category == 'centre']
gdf_pois.index = gdf_pois.index.map(int)
x, y = gdf_pois.lon, gdf_pois.lat
network.set_pois(category='POIs', maxdist = 2500, maxitems=len(gdf_pois), x_col=gdf_pois['lon'], y_col=gdf_pois['lat'])
print("Pandana network completed")
gdf_pois["node_ids"] = network.get_node_ids(x, y)

In [ ]:
#Buffer mismatch error arise when the quantity variable is an int64 and not a float...stupid bug !!!
network.set(gdf_pois["node_ids"], variable=gdf_pois.quantity, name="density")
density = network.aggregate(2500, type="sum", decay="linear", name="density")
nearest = network.nearest_pois(distance=2500, category='Screening center', num_pois=5)
access = [density,nearest]
dict_density['screening center'] = access

In [ ]:
df_density = pd.DataFrame(dict_density['screening center'][1].index)
df_density = df_density.set_index('osmid')
df_density['poi_density'] = dict_density['screening center'][0]
df_density['poi_nearest'] = dict_density['screening center'][1][1]
df_density['poi_5nearest'] = dict_density['screening center'][1][5]

In [ ]:
df_density.index = df_density.index-9990000000

In [ ]:
people_density = people_geom.join(df_density)

In [ ]:
gdf_geom_density = pd.merge(gdf_geom_ok,people_density[['id','poi_density','poi_nearest','poi_5nearest']],how = 'left',left_on = 'geom_id',right_on = 'id').drop('id',axis = 1)